# Modules

In [5]:
import os
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
import configparser
import realstate_utils as rs


In [6]:
config = configparser.ConfigParser()
config.read('config.ini')

path = config.get('PATH', 'PATH_DATA')
url = config.get('URL', 'URL')
page = int(config.get('PAGE', 'page'))

# Processing

In [11]:
spark = pyspark.sql.SparkSession.builder \
    .appName("Real_state") \
    .getOrCreate()

In [4]:
spark.active()

In [16]:
# Pour définir la valeur si nécessaire
os.environ["PYSPARK_PYTHON"] = "/c/Users/bakar/anaconda3/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/c/Users/bakar/anaconda3/python.exe"



In [17]:
print("PYSPARK_PYTHON:", os.getenv("PYSPARK_PYTHON"))
print("PYSPARK_DRIVER_PYTHON:", os.getenv("PYSPARK_DRIVER_PYTHON"))

PYSPARK_PYTHON: /c/Users/bakar/anaconda3/python.exe
PYSPARK_DRIVER_PYTHON: /c/Users/bakar/anaconda3/python.exe


In [11]:
print(url)

https://www.citya.com/annonces/vente/appartement,maison?sort=b.dateMandat&direction=desc


In [9]:
data = rs.scrapping(url,page)
data

c:\Users\bakar\OneDrive\Bureau\Projets_Perso\NukuData\Real-estate-prices-etl\NukuData\ETL-real-estate-prices\Scripts\realstate_utils.py:25: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 25 of the file c:\Users\bakar\OneDrive\Bureau\Projets_Perso\NukuData\Real-estate-prices-etl\NukuData\ETL-real-estate-prices\Scripts\realstate_utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(html, "html")


[{'id': '1_0',
  'title': 'Maison                     4 pièces 93m²',
  'description': "Brissay-Choigny : maison 4 pièces (93 m²) à vendre - EXCLUSIVITÉ-EN BON ÉTAT GÉNÉRAL\nBERNOVILLE IMMOBILIER (Tergnier) vous propose cette maison [...]\n                            de 4 pièces 93 m² et de 886 m² de terrain à vendre à Brissay-Choigny (02240). Son intérieur dispose d'une entrée, d'une pièce à vivre, de deux chambres de 14 m² chacune, d'une cuisine aménagée de 21 m² et d'un bureau. Il compte aussi une salle d'eau et des toilettes.\nElle est en bon état général. Un garage est prévu pour garer votre véhicule.\nOn trouve une école élémentaire dans le quartier. Côté transports en commun, il y a cinq gares à moins de 10 minutes en voiture. On trouve un accès à l'autoroute A26 à 8 km. On trouve un commerce à quelques minutes à peine.\nContactez nos conseillers pour obtenir de plus amples renseignements sur cette maison en vente à Brissay-Choigny.  Honoraires à la charge du vendeur. \nVous pou

In [19]:
df = spark.createDataFrame(data)

In [ ]:
df.show(1,False)

: 

In [15]:

import os

# Pour afficher la valeur
print("PYSPARK_PYTHON:", os.getenv("PYSPARK_PYTHON"))
print("PYSPARK_DRIVER_PYTHON:", os.getenv("PYSPARK_DRIVER_PYTHON"))

PYSPARK_PYTHON: /c/Users/bakar/AppData/Local/Programs/Python/Python312/python
PYSPARK_DRIVER_PYTHON: /c/Users/bakar/AppData/Local/Programs/Python/Python312/python


In [6]:
df.show()

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|    220|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|    250|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|    190|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|    190|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|    210|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|    210|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|    210|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|    210|«Une occasion à n...|
|Nouvelle promotio...|  8|1861800| 5.5|    210|«Une opportunité ...|
|Érigé sur une jol...|  9|1181280| 5.5|    180|«Environnement id...|
|Projekt In dieser...| 10|1919580| 5.5|    260|«RÄBBERG - AUSSIC...|
|Cette nouvelle pr...| 11|1206960|

In [7]:
df = df.withColumn("id", df["id"].cast(IntegerType())) \
    .withColumn("price", df["price"].cast(IntegerType())) \
    .withColumn("room", df["room"].cast(DoubleType())) \
    .withColumn("surface", df["surface"].cast(DoubleType()))
df.printSchema()
df.show()

root
 |-- description: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- room: double (nullable = true)
 |-- surface: double (nullable = true)
 |-- title: string (nullable = true)

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|  220.0|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|  250.0|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|  190.0|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|  190.0|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|  210.0|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|  210.0|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|  210.0|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|  210.0|«Une occasion à n...|
|Nouvel

In [8]:
json_files = [os.path.join(PATH_DATA_RAW, f) for f in os.listdir(PATH_DATA_RAW) if f.endswith(".json")]

# Initialisez un DataFrame vide
concatenated_df = None

# Parcourez la liste des fichiers JSON et lisez-les, puis concaténez-les
for json_file in json_files:
    # Lisez chaque fichier JSON dans un DataFrame
    temp_df = spark.read.json(json_file)
    
    # Si le DataFrame temporaire n'est pas vide, concaténez-le
    if not temp_df.isEmpty():
        if concatenated_df is None:
            concatenated_df = temp_df
        else:
            concatenated_df = concatenated_df.union(temp_df)

# Si "concatenated_df" n'est pas vide, affichez le schéma et les données
if concatenated_df is not None and not concatenated_df.isEmpty():
    concatenated_df.printSchema()
    concatenated_df.show()

root
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- price: string (nullable = true)
 |-- room: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- title: string (nullable = true)

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|    220|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|    250|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|    190|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|    190|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|    210|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|    210|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|    210|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|    210|«Une occasion à n...|
|Nouvelle p

In [9]:
concatenated_df.count()

1008